# ADMISSIONS & OCCUPANCY

## Functions

In [1]:
from cuhvid.admissions import get_admissions
from cuhvid.bed_occupancy import get_bed_occupancy
from cuhvid.triggers import get_rolling_mean

## Initial parameter values

In [2]:
_init_vals = dict(
    alpha    = 6.57,
    loc      = 0,
    scale    = 6.56,
    peak_adm = 12,
    sd_coeff = 0.3,
    sd_const = 0.6,
    LoS_GIM  = 9,    # ppt 9    , best 9
    LoS_ICU  = 18,   # ppt 23   , best 18
    frac_ICU = 0.19, # ppt 0.25 , best 0.19
)

## Data import

In [3]:
import pandas as pd
import numpy as np

path = '../Data/RAWDATA_Covid_Admissions.xlsx'
df_act = pd.read_excel(path, converters={
    'Date': pd.to_datetime,
    'Admissions': np.float64
})

## Plotting

In [4]:
from ipywidgets import widgets
import plotly.graph_objects as go


# ------------------------------------------------------------------------------
# Widgets
# ------------------------------------------------------------------------------
reset = widgets.Button(
    description='Reset',
    button_style='warning',
    tooltip='Reset sliders to default values',
    icon='refresh'
)

regen = widgets.Button(
    description='Randomize',
    button_style='',
    tooltip='Regenerate random series keeping all slider values',
    icon='refresh'
)

alpha = widgets.FloatSlider(
    value=_init_vals['alpha'],
    min=0.1,
    max=10,
    step=0.05,
    description='Shape:',
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

loc = widgets.FloatSlider(
    value=_init_vals['loc'],
    min=-10,
    max=10,
    step=0.1,
    description='Shift:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

scale = widgets.FloatSlider(
    value=_init_vals['scale'],
    min=0.1,
    max=10,
    step=0.01,
    description='Scale:',
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

peak_adm = widgets.IntSlider(
    value=_init_vals['peak_adm'],
    min=1,
    max=50,
    step=1,
    description='Peak adm.:',
    continuous_update=True,
)

sd_coeff = widgets.FloatSlider(
    value=_init_vals['sd_coeff'],
    min=0,
    max=1,
    step=0.01,
    description='σ coeff.:',
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

sd_const = widgets.FloatSlider(
    value=_init_vals['sd_const'],
    min=0,
    max=1,
    step=0.05,
    description='σ const.:',
    continuous_update=True,
    readout=True,
    readout_format='.2f',
)

sd_options = widgets.ToggleButtons(
    options=['Constant', 'Linear Correlation'],
    description='Variability:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Constant variability', 'Variability linearly related to daily admissions'],
    value='Linear Correlation'
)

LoS_GIM = widgets.IntSlider(
    value=_init_vals['LoS_GIM'],
    min=1,
    max=50,
    step=1,
    description='LoS GIM:',
    continuous_update=True,
)

LoS_ICU = widgets.IntSlider(
    value=_init_vals['LoS_ICU'],
    min=1,
    max=50,
    step=1,
    description='LoS ICU:',
    continuous_update=True,
)

frac_ICU = widgets.FloatSlider(
    value=_init_vals['frac_ICU'],
    min=0,
    max=1,
    step=0.01,
    description='% ICU:',
    continuous_update=True,
    readout=True,
    readout_format='.0%',
)

no_days = widgets.IntSlider(
    value=7,
    min=1,
    max=10,
    step=1,
    description='No days:',
    continuous_update=True,
)

center = widgets.Checkbox(
    value=False,
    description='Centred rolling mean',
    disabled=False
)


# ------------------------------------------------------------------------------
# Layout
# ------------------------------------------------------------------------------
layout = go.Layout(
    title='Covid admissions: Daily',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. admissions'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout2 = go.Layout(
    title='Covid admissions: Cumulative',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'Cumulative no. admissions'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout3 = go.Layout(
    title='Covid bed occupancy',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. beds'
    },
    legend=dict(
        x=1,
        y=1
    )
)

layout4 = go.Layout(
    title='Admissions trigger: Daily',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'No. admissions'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)

layout5 = go.Layout(
    title='Net intake trigger: Daily',
    xaxis={
        'title': 'Day',
        'tickmode': 'linear',
        'tick0': 0,
        'dtick': 7
    },
    yaxis={
        'title': 'Net patient intake'
    },
    legend=dict(
        x=0,
        y=-0.5
    )
)


# ------------------------------------------------------------------------------
# Functions to link widget values to dataframe functions
# ------------------------------------------------------------------------------
def get_admissions_params():
    """
    Returns dictionary of values of parameters for get_admissions()
    
    e.g. usage: 
        get_admissions(**get_admissions_params())
    """
    params = dict(alpha=alpha.value,
                  loc=loc.value,
                  scale=scale.value,
                  peak_adm=peak_adm.value,
                  sd_coeff=sd_coeff.value,
                  sd_const=sd_const.value)
    return params

def get_occupancy_params():
    """
    Returns dictionary of values of parameters for get_bed_occupancy()
    
    e.g. usage: 
        get_bed_occupancy(df, **get_occupancy_params())
    """
    params = dict(LoS_GIM=LoS_GIM.value,
                  LoS_ICU=LoS_ICU.value,
                  frac_ICU=frac_ICU.value)
    return params

def get_rolling_params():
    """
    Returns dictionary of values of parameters for get_rolling_mean()
    
    e.g. usage: 
        get_rolling_mean(df, columns, **get_rolling_params())
    """
    params = dict(no_days=no_days.value,
                  center=center.value)
    return params


# ------------------------------------------------------------------------------
# Populate dataframe
# ------------------------------------------------------------------------------
df_gen = get_admissions(**get_admissions_params())
df_gen = get_bed_occupancy(df_gen, **get_occupancy_params())

rolling_mean_columns = ['y_gen', 'net_intake_gen']
df_gen = get_rolling_mean(df_gen, rolling_mean_columns, **get_rolling_params())


# ------------------------------------------------------------------------------
# Set up plot traces
# ------------------------------------------------------------------------------

## --------------- Daily data ---------------
data = []
data.append(go.Scatter(
    name = 'Probability density function',
    x = df_gen.index,
    y = df_gen.y_lower,
    mode='lines',
    line=dict(width=0),
    showlegend=False
))
data.append(go.Scatter(
    name = 'Probability density function',
    x = df_gen.index,
    y = df_gen.y,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
    fill='tonexty',
    fillcolor='rgba(0, 176, 246, 0.3)',
))
data.append(go.Scatter(
    name = 'Probability density function',
    x = df_gen.index,
    y = df_gen.y_upper,
    mode='lines',
    line=dict(width=0),
    fill='tonexty',
    fillcolor='rgba(0, 176, 246, 0.3)',
    showlegend=False
))
data.append(go.Scatter(
    name = 'Generated series',
    x = df_gen.index,
    y = df_gen.y_gen,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = df_gen.index,
    y = df_act.Admissions,
    mode='lines',
    line=dict(color='rgb(98, 76, 171)', dash='dot'),
))

## --------------- Cumulative data ---------------
data2 = []
data2.append(go.Scatter(
    name = 'Probability density function',
    x = df_gen.index,
    y = df_gen.y.cumsum(),
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
))
data2.append(go.Scatter(
    name = 'Generated series',
    x = df_gen.index,
    y = df_gen.y_gen.cumsum(),
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data2.append(go.Scatter(
    name = 'Admissions (first wave)',
    x = df_gen.index,
    y = df_act.Admissions.cumsum().values,
    mode='lines',
    line=dict(color='rgb(98, 76, 171)', dash='dot'),
))

## --------------- Bed occupancy data ---------------
data3 = []
data3.append(go.Scatter(
    name = 'Generated GIM',
    x = df_gen.index,
    y = df_gen.GIM_gen,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)')
))
data3.append(go.Scatter(
    name = 'Actual GIM',
    x = df_gen.index,
    y = df_act['GIM bed occupancy'],
    mode='lines',
    line=dict(color='rgb(98, 76, 171)')
))
data3.append(go.Scatter(
    name = 'Fitted GIM',
    x = df_gen.index,
    y = df_gen.GIM,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)'),
))
data3.append(go.Scatter(
    name = 'Generated ICU',
    x = df_gen.index,
    y = df_gen.ICU_gen,
    mode='lines',
    line=dict(color='rgb(166, 139, 165)', dash='dot')
))
data3.append(go.Scatter(
    name = 'Actual ICU',
    x = df_gen.index,
    y = df_act['ICU bed occupancy'],
    mode='lines',
    line=dict(color='rgb(98, 76, 171)', dash='dot'),
))
data3.append(go.Scatter(
    name = 'Fitted ICU',
    x = df_gen.index,
    y = df_gen.ICU,
    mode='lines',
    line=dict(color='rgb(0, 176, 246)', dash='dot'),
))

## --------------- Admissions trigger ---------------
data4 = []
data4.append(
    go.Scatter(
        name = 'Admissions - Daily',
        x = df_gen.index,
        y = df_gen.y_gen,
        mode = 'lines',
        opacity = 0.5
    )
)
data4.append(
    go.Scatter(
        name = f'Admissions - ({no_days.value} day rolling mean)',
        x = df_gen.index,
        y = df_gen.y_gen_rm
    )
)

## --------------- Net intake trigger ---------------
data5 = []
data5.append(
    go.Scatter(
        name = 'Net intake - Daily',
        x = df_gen.index,
        y = df_gen.net_intake_gen,
        mode = 'lines',
        opacity = 0.5
    )
)
data5.append(
    go.Scatter(
        name = f'Net intake - ({no_days.value} day rolling mean)',
        x = df_gen.index,
        y = df_gen.net_intake_gen_rm
    )
)


# ------------------------------------------------------------------------------
# Plot
# ------------------------------------------------------------------------------
g = go.FigureWidget(data=data, layout=layout)
g2 = go.FigureWidget(data=data2, layout=layout2)
g3 = go.FigureWidget(data=data3, layout=layout3)
g4 = go.FigureWidget(data=data4, layout=layout4)
g5 = go.FigureWidget(data=data5, layout=layout5)


# ------------------------------------------------------------------------------
# Responsive functions
# ------------------------------------------------------------------------------

## --------------- Replot on change ---------------
def response(change):
    # ..... Repopulate dataframe .....
    df_gen = get_admissions(**get_admissions_params())
    df_gen = get_bed_occupancy(df_gen, **get_occupancy_params())
    df_gen = get_rolling_mean(df_gen, rolling_mean_columns, **get_rolling_params())

    with g.batch_update():
        g.data[0].y = df_gen.y_lower
        g.data[1].y = df_gen.y
        g.data[2].y = df_gen.y_upper
        g.data[3].y = df_gen.y_gen
        
    with g2.batch_update():
        g2.data[0].y = df_gen.y.cumsum()
        g2.data[1].y = df_gen.y_gen.cumsum()
        
    with g3.batch_update():
        g3.data[0].y = df_gen.GIM_gen
        g3.data[2].y = df_gen.GIM
        g3.data[3].y = df_gen.ICU_gen
        g3.data[5].y = df_gen.ICU
    
    with g4.batch_update():
        g4.data[0].y = df_gen.y_gen
        g4.data[1].y = df_gen.y_gen_rm
        
    with g5.batch_update():
        g5.data[0].y = df_gen.net_intake_gen
        g5.data[1].y = df_gen.net_intake_gen_rm
    
    return

## --------------- Reset to default values ---------------
def reset_values(change):
    alpha.value = _init_vals['alpha']
    loc.value = _init_vals['loc']
    scale.value = _init_vals['scale']
    peak_adm.value = _init_vals['peak_adm']
    sd_coeff.value = _init_vals['sd_coeff']
    sd_const.value = _init_vals['sd_const']
    LoS_GIM.value = _init_vals['LoS_GIM']
    LoS_ICU.value = _init_vals['LoS_ICU']
    frac_ICU.value = _init_vals['frac_ICU']
    
    response(None)
    return

## --------------- Variability options ---------------
def variability(change):
    if change['new'] == 'Constant':
        sd_coeff.value = 0
        sd_const.value = 0.75
        sd_coeff.disabled = True
    elif change['new'] == 'Linear Correlation':
        sd_coeff.value = _init_vals['sd_coeff']
        sd_const.value = _init_vals['sd_const']
        sd_coeff.disabled = False
    return


# ------------------------------------------------------------------------------
# Link widgets to functions
# ------------------------------------------------------------------------------
reset.on_click(reset_values)
regen.on_click(response)
sd_options.observe(variability, names='value')
alpha.observe(response, names='value')
loc.observe(response, names='value')
scale.observe(response, names='value')
peak_adm.observe(response, names='value')
sd_coeff.observe(response, names='value')
sd_const.observe(response, names='value')
LoS_GIM.observe(response, names='value')
LoS_ICU.observe(response, names='value')
frac_ICU.observe(response, names='value')
no_days.observe(response, names='value')
center.observe(response, names='value')


# ------------------------------------------------------------------------------
# Display widgets & plots
# ------------------------------------------------------------------------------

## --------------- Widgets ---------------
### ..... Admissions .....
gamma_control = widgets.VBox([
    widgets.HTML(value='<h4>Curve parameters</h4>'),
    alpha,
    loc,
    scale,
    peak_adm,
])
variability_control = widgets.VBox([
    widgets.HTML(value='<h4>Variability parameters</h4>'),
    sd_coeff,
    sd_const,
    sd_options
])
button_control = widgets.VBox([
    reset,
    regen
])

admissions_control = widgets.HBox([
    gamma_control,
    variability_control,
    button_control
])
admissions_plots = widgets.HBox([
    g,
    g2
])

admissions = widgets.VBox([
    widgets.HTML(value='<h3>Patient Admissions</h3>'),
    admissions_control,
    admissions_plots
])

### ..... Bed occupancy .....
occ_control = widgets.VBox([
    widgets.HTML(value='<h4>Bed occupancy parameters</h4>'),
    LoS_GIM,
    LoS_ICU,
    frac_ICU
])

occupancy = widgets.VBox([
    widgets.HTML(value='<h3>Bed Occupancy</h3>'),
    widgets.HBox([
        occ_control,
        g3
    ])
])

### ..... Triggers .....
trigger_control = widgets.VBox([
    no_days,
    center
])

triggers = widgets.VBox([
    widgets.HTML(value='<h3>Triggers</h3>'),
    trigger_control,
    widgets.HBox([
        g4,
        g5
    ])
])

## --------------- Display ---------------
widgets.VBox([
    admissions,
    occupancy,
    triggers
])